In [17]:
import json
from time import sleep
import requests
import pandas as pd

In [56]:
def get_user_by_rank(page):
    """
     해당 페이지의 유저 정보를 받아옵니다.
     유저는 랭크순으로 정렬되어 있습니다.
    """
    try:
        allData = []
        url = f"https://solved.ac/api/v3/ranking/tier?page={page}"
        r_profile = requests.get(url)
        if r_profile.status_code == requests.codes.too_many_requests: 
                return False
        profiles = json.loads(r_profile.content.decode('utf-8'))["items"]
        for profile in profiles:
            allData.append(profile)
        return allData
    
    except Exception as e:    
        print(f'{page}에서 예외가 발생했습니다.', e)

def run_user_by_rank(start_page,end_page):
    """
     start_page부터 end_page까지의 사용자 정보를 받아옵니다.
     만약 too many request 에러가 발생하면 500초 동안 동작을 일시정지한 후 재개합니다.
    """
    df = pd .DataFrame()
    page = start_page
    while page <= end_page:
        allData = get_user_by_rank(page)
        if allData == False: 
            print(f"{page}번째 처리 중 Error 429 발생")
            sleep(500)
        else:
            for data in allData:
                df = df.append(data,ignore_index = True)
            if page % 50 == 0:
                print(f"{page} Completed")
            page += 1
    df.to_csv("result/user_data.csv")

In [99]:
url = f"https://solved.ac/api/v3/ranking/tier?page={page}"
r_profile = requests.get(url)        
count = json.loads(r_profile.content.decode('utf-8'))["count"]
print("총 페이지 수는 ",count//50 + 1,"입니다")

총 페이지 수는  2424 입니다


In [102]:
run_user_by_rank(1,3)